In [30]:
import pandas as pd
import numpy as np

In [7]:
ds1 = pd.read_csv('data/21600-0001-Data.tsv',sep='\t')
ds5 = pd.read_csv('data/21600-0005-Data.tsv',sep='\t')
ds8 = pd.read_csv('data/21600-0008-Data.tsv',sep='\t')
ds22 = pd.read_csv('data/21600-0022-Data.tsv',sep='\t')

/Users/lucasrosenblatt/opt/miniconda3/envs/heuristic_fairness/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6,12,55,124,125,167,195,196,197,198,336,356,513,522,523,743,745,746,832,833,837,838,839,846,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,956,957,964,969,970,972,976,977,979,981,982,984,1000,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1063,1065,1068,1069,1070,1071,1074,1075,1076,1077,1078,1081,1082,1083,1084,1085,1087,1088,1089,1109,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1174,1175,1176,1177,1180,1181,1182,1183,1184,1187,1188,1189,1190,1191,1209,1211,1212,1225,1226,1227,1228,1229,1230,1252,1253,1254,1255,1256,1257

In [69]:
df_map = {'ds1': ds1, 'ds5': ds5, 'ds8': ds8, 'ds22': ds22}
column_map = {
    'ds1': {
        'H1TO32': 'marijuana_1',
        'H1TO15': 'alcohol_1',
        'H1TO40': 'drugs_1',
        'H1DS2': 'behavior_1',
        'H1GI18': 'attendance_1',
        'H1TO7': 'smoking_1'
    },
    'ds5': {
        'H2TO46': 'marijuana_2',
        'H2TO19': 'alcohol_2',
        'H2TO58': 'drugs_2',
        'H2DS2': 'behavior_2',
        'H2GI6': 'attendance_2',
        'H2TO7': 'smoking_2'
    },
    'ds8': {
        'H3TO110': 'marijuana_3',
        'H3TO38': 'alcohol_3',
        'H3TO117': 'drugs_3',
        'H3DS1': 'behavior_3',
        'H3ED23': 'attendance_3',
        'H3TO10': 'smoking_3'
    },
    'ds22': {
        'H4TO71': 'marijuana_4',
        'H4TO35': 'alcohol_4',
        'H4TO65E': 'drugs_4',
        'H4DS1': 'behavior_4',
        'H4ED6': 'attendance_4',
        'H4TO6': 'smoking_4'
    },
}

In [70]:
dfs = []
new_df_map = {}
for wave, cols in column_map.items():
    df = df_map[wave]
    df = df[['AID'] + list(cols.keys())]
    df = df.rename(cols, axis=1)
    df = df.set_index('AID')
    dfs.append(df)
    new_df_map[wave] = df
    
full_df = pd.concat(dfs, axis=1, join='inner').reset_index()

For small categories, these are missing values
- 6  
- 7 (legitimate skip)
- 8

For medium categories, these are missing values
- 96  
- 97 (legitimate skip)
- 98

For large categories, these are missing values
- 996 
- 997 (legitimate skip)
- 998

In [71]:
# Paper says n=6504, first wave table
new_df_map['ds1'].describe()

,marijuana_1,alcohol_1,behavior_1,attendance_1,smoking_1
count,6504.000000,6504.000000,6504.000000,6504.000000,6504.000000
mean,745.144834,46.892066,0.291205,0.983702,74.107011
std,431.505517,45.795986,0.851450,0.194604,39.438188
min,0.000000,1.000000,0.000000,0.000000,0.000000
25%,60.000000,5.000000,0.000000,1.000000,30.000000
50%,997.000000,7.000000,0.000000,1.000000,97.000000
75%,997.000000,97.000000,0.000000,1.000000,97.000000
max,999.000000,98.000000,9.000000,8.000000,98.000000


In [72]:
# Need to fix some broken data here
new_df_map['ds1']['drugs_1'] = pd.to_numeric(new_df_map['ds1']['drugs_1'], errors='coerce')

In [73]:
# Check some stats from table
new_df_map['ds1'].loc[new_df_map['ds1']['marijuana_1'] > 995, 'marijuana_1'] = np.NaN
new_df_map['ds1'].loc[new_df_map['ds1']['alcohol_1'] > 95, 'alcohol_1'] = np.NaN
new_df_map['ds1'].loc[new_df_map['ds1']['drugs_1'] > 95, 'drugs_1'] = np.NaN
new_df_map['ds1'].loc[new_df_map['ds1']['attendance_1'] > 5, 'attendance_1'] = np.NaN
new_df_map['ds1'].loc[new_df_map['ds1']['behavior_1'] > 5, 'behavior_1'] = np.NaN
new_df_map['ds1'].loc[new_df_map['ds1']['smoking_1'] > 95, 'smoking_1'] = np.NaN
new_df_map['ds1'].describe()
# Yay, these all align!

,marijuana_1,alcohol_1,drugs_1,behavior_1,attendance_1,smoking_1
count,1655.000000,3545.000000,6402.000000,6443.000000,6501.000000,1653.000000
mean,7.219940,5.067419,1.103405,0.229241,0.980772,6.921355
std,35.201338,1.493296,3.899566,0.559196,0.137335,8.178834
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.000000,0.000000,0.000000,1.000000,1.000000
50%,1.000000,5.000000,0.000000,0.000000,1.000000,4.000000
75%,4.000000,6.000000,0.000000,0.000000,1.000000,10.000000
max,800.000000,7.000000,18.000000,3.000000,1.000000,89.000000


In [74]:
# Only 3342 consistent IDs throughout experiment?
full_df

,AID,marijuana_1,alcohol_1,drugs_1,behavior_1,attendance_1,smoking_1,marijuana_2,alcohol_2,drugs_2,...,drugs_3,behavior_3,attendance_3,smoking_3,marijuana_4,alcohol_4,drugs_4,behavior_4,attendance_4,smoking_4
0,57101310,997,5,0,0,1,8,0,4,0,...,0,0,0,10,97,2,0,0,0,4
1,57103869,998,97,98,0,1,1,1,3,0,...,0,0,0,997,1,97,0,0,0,2
2,57109625,0,5,0,1,1,97,0,4,0,...,1,1,0,997,6,5,1,0,0,997
3,57111071,997,97,0,0,1,97,997,97,0,...,0,1,1,997,97,4,1,0,0,997
4,57113943,997,97,0,0,1,97,997,97,0,...,0,0,0,997,0,1,0,0,0,997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,99718925,997,97,0,0,1,97,997,97,0,...,0,0,0,20,97,97,0,0,0,20
3338,99718944,997,97,0,0,1,97,997,97,0,...,0,0,1,997,97,4,0,0,1,997
3339,99719378,997,97,0,0,1,97,1,97,0,...,0,0,1,997,97,4,0,0,0,4
3340,99719930,997,97,0,0,1,97,997,97,0,...,0,0,1,997,97,1,0,0,0,997
